## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets
import csv


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
# city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df = pd.read_csv("C:\\Users\chuba\Class\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Leander,US,30.5788,-97.8531,93.27,54,0,5.01,clear sky
1,1,Torbay,CA,47.6666,-52.7314,57.27,91,90,9.22,overcast clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,53.28,66,100,2.30,overcast clouds
3,3,Seddon,AU,-37.8068,144.8916,59.65,74,75,12.66,broken clouds
4,4,Mataura,NZ,-46.1927,168.8643,48.47,87,100,2.24,light rain


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 85


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Mahibadhoo,MV,3.7833,72.9667,82.24,77,100,9.31,moderate rain
12,12,Catalina,US,32.5056,-110.9211,80.58,71,100,1.19,overcast clouds
20,20,Pandan Niog,PH,6.2282,120.5631,83.52,76,100,11.39,overcast clouds
30,30,Kavieng,PG,-2.5744,150.7967,83.17,70,40,4.14,scattered clouds
32,32,Gazanjyk,TM,39.2446,55.5154,80.96,73,0,10.49,clear sky
39,39,Nishihara,JP,26.1842,127.7558,83.86,84,75,12.66,broken clouds
42,42,Hithadhoo,MV,-0.6000,73.0833,80.46,76,100,5.70,moderate rain
45,45,Kahului,US,20.8947,-156.4700,83.46,89,32,11.99,scattered clouds
50,50,Ugoofaaru,MV,5.6667,73.0000,81.54,78,100,16.87,overcast clouds
53,53,Kokopo,PG,-4.3520,152.2633,83.62,68,78,1.39,broken clouds


In [11]:
# 4a. Determine if there are any empty rows. 
# Get the columns and rows that are not null
preferred_cities_df.count()

City_ID                104
City                   104
Country                104
Lat                    104
Lng                    104
Max Temp               104
Humidity               104
Cloudiness             104
Wind Speed             104
Current Description    104
dtype: int64

In [12]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Mahibadhoo,MV,3.7833,72.9667,82.24,77,100,9.31,moderate rain
12,12,Catalina,US,32.5056,-110.9211,80.58,71,100,1.19,overcast clouds
20,20,Pandan Niog,PH,6.2282,120.5631,83.52,76,100,11.39,overcast clouds
30,30,Kavieng,PG,-2.5744,150.7967,83.17,70,40,4.14,scattered clouds
32,32,Gazanjyk,TM,39.2446,55.5154,80.96,73,0,10.49,clear sky
...,...,...,...,...,...,...,...,...,...,...
683,683,Thinadhoo,MV,0.5333,72.9333,81.93,69,99,2.01,overcast clouds
684,684,Elizabeth City,US,36.2946,-76.2511,84.02,80,69,1.01,broken clouds
689,689,Fukue,JP,32.6881,128.8419,80.96,83,75,6.91,broken clouds
702,702,Campoverde,PE,-8.4753,-74.8058,82.09,56,11,2.33,few clouds


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Mahibadhoo,MV,82.24,moderate rain,3.7833,72.9667,
12,Catalina,US,80.58,overcast clouds,32.5056,-110.9211,
20,Pandan Niog,PH,83.52,overcast clouds,6.2282,120.5631,
30,Kavieng,PG,83.17,scattered clouds,-2.5744,150.7967,
32,Gazanjyk,TM,80.96,clear sky,39.2446,55.5154,
39,Nishihara,JP,83.86,broken clouds,26.1842,127.7558,
42,Hithadhoo,MV,80.46,moderate rain,-0.6000,73.0833,
45,Kahului,US,83.46,scattered clouds,20.8947,-156.4700,
50,Ugoofaaru,MV,81.54,overcast clouds,5.6667,73.0000,
53,Kokopo,PG,83.62,broken clouds,-4.3520,152.2633,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
           
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")
        
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.

import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Mahibadhoo,MV,82.24,moderate rain,3.7833,72.9667,Omadhoo Inn
12,Catalina,US,80.58,overcast clouds,32.5056,-110.9211,Miraval Arizona Resort & Spa
30,Kavieng,PG,83.17,scattered clouds,-2.5744,150.7967,Nusa Island Retreat
32,Gazanjyk,TM,80.96,clear sky,39.2446,55.5154,Railway motel
39,Nishihara,JP,83.86,broken clouds,26.1842,127.7558,Minshuku Agaihama


In [17]:
hotel_df.columns

Index(['City', 'Country', 'Max Temp', 'Current Description', 'Lat', 'Lng',
       'Hotel Name'],
      dtype='object')

In [19]:
# 8a. Create the output File (CSV) ## CHECK FOLDER LOCATION

# output_data_file2 = "Vacation_Search/WeatherPy_vacation_.csv"
output_data_file2 = "Vacation_Search/WeatherPy_vacation_.csv"

# 8b. Export the City_Data into a csv 
hotel_df.to_csv(output_data_file2, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig





Figure(layout=FigureLayout(height='420px'))